# gdir_preprocessing.ipynb

Brandon S. Tober<br>

23JUL2025<br>

This notebook demonstrates how OGGM glacier directories can be compressed after some preprocessing.  Say we perform bed inversion and then dynamical spinup, and wish to store a set of compressed glacier directories afterwards.

First, run OGGMs ice thickness/bed inversion using PyGEM's mass balance model

```bash
run_inversion -rgi_region01 1 -ncores 6
```

Second, perform dyncamical spinup, using PyGEM's mass balance model

```bash
run_spinup -rgi_region01 1 -target_yr 2000 -ncores 6 
```
 
Both the above example steps will run for all glaciers in RGI region 1. for spinup, we could instead pass a list of glaciers to run (`-rgi_glac_number_fn` argument), or specify particular glaciers (`-rgi_glac_number` argument).

After we perform these steps, we may wish to compress out preprocessed glacier directories.

OGGM's `utils.gdir_to_tar()` method will compress individual gdirs (given a list of gdirs). Then, OGGM's `utils.base_dir_to_tar()` method will bundle individual gdir tars to a base-level directory tar.

```python

import oggm.cfg as cfg
from oggm import utils, workflow, tasks
import pygem.setup.config as config
import geopandas as gpd

# read pygem config
pygem_prms = config.read_config()
# Initialize OGGM and set up the default run parameters
cfg.initialize(logging_level='WARNING')
# set working directory
cfg.PATHS['working_dir'] =f"{pygem_prms['root']}/{pygem_prms['oggm']['oggm_gdir_relpath']}"
cfg.PARAMS['border'] = 80
rgi_ids = gpd.read_file(utils.get_rgi_region_file('01', version='62'))['RGIId'].tolist()
# probably want to use multiprocessing
cfg.PARAMS['use_multiprocessing']=True
# initialize gdirs from working_dir
gdirs = workflow.init_glacier_directories(rgi_ids);
# tar the individual glacier directories first
workflow.execute_entity_task(utils.gdir_to_tar, gdirs, delete=True);
# then tar the bundles
utils.base_dir_to_tar(cfg.PATHS['working_dir'], delete=True);

# now, say we wish to reload a set of glacier directories:
gdirs = workflow.init_glacier_directories(rgi_ids, from_tar=True);
```

Say we want to run L4p preprocessing (level 4 "PyGEM") on all RGI regions:

```bash
# define regions of interest
regions=($(seq 1 19))
# how many cores to use
ncores=128
# run inversion script
run_inversion -rgi_region01 "${regions[@]}" -ncores $ncores
# run spinup script
run_spinup -rgi_region01 "${regions[@]}" -ncores $ncores
# compress gdirs
compress_gdirs -rgi_region01 "${regions[@]}" -ncores $ncores
```